# 1.0 Install Packages and Import Libraries

In [1]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
 

In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login
import wandb

# Login to HuggingFace
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) y


In [3]:
# Login to weights and biases (to track training metrics)
wandb.login()
%env WANDB_PROJECT=Fine-Tune-QLoRA

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=Fine-Tune-QLoRA


# 2.0 Load Dataset

In [4]:
# Load the dataset
dataset = load_dataset("microsoft/orca-math-word-problems-200k") #("hendrycks/competition_math")

# # Drop the 'level' and 'type' columns from the dataset
# dataset = dataset.remove_columns(['level', 'type'])

train_testvalid = dataset['train'].train_test_split(test_size=0.2)

# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

from datasets.dataset_dict import DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

README.md:   0%|          | 0.00/6.91k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/84.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200035 [00:00<?, ? examples/s]

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 160028
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 20004
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 20003
    })
})


In [6]:
print(dataset['train'][0])

{'question': 'the amount of time that three people worked on a special project was in the ratio of 3 to 5 to 6 . if the project took 140 hours , how many more hours did the hardest working person work than the person who worked the least ?', 'answer': "Let's denote the amount of time the three people worked as 3x, 5x, and 6x, respectively. The total amount of time worked by all three people is the sum of these amounts, which is equal to 140 hours. So we have:\n\n3x + 5x + 6x = 140\n\nCombining like terms, we get:\n\n14x = 140\n\nTo find the value of x, we divide both sides of the equation by 14:\n\nx = 140 / 14\nx = 10\n\nNow we can find the amount of time each person worked:\n\nThe first person worked 3x = 3 * 10 = 30 hours.\nThe second person worked 5x = 5 * 10 = 50 hours.\nThe third person worked 6x = 6 * 10 = 60 hours.\n\nThe hardest working person worked 60 hours, and the person who worked the least worked 30 hours. To find out how many more hours the hardest working person worked

# 3.0 Configure Bits and Bytes

In [7]:
# Ensure the computation uses 16-bit floating-point (reduce memory usage, speed up training)
compute_dtype = getattr(torch, "float16")

# Configure Bits and Bytes to load the model in 4-bit (quantized)
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, # Load the weights in 4 bit
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype, # Uses 16-bit floating-point (float16)
        bnb_4bit_use_double_quant=True, # Enable double quantization
    )

# 4.0 Load the Pretrained Model in 4-bit (Quantanized)

In [8]:
# Load the pretrained model, 'meta-llama/Llama-3.2-1B-Instruct' required authorization
base_model_name = 'Qwen/Qwen2.5-0.5B-Instruct'
device_map = "auto" #{"": 0}
base_model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config, # To load in 4-bit and double quantization
                                                      trust_remote_code=True,
                                                      use_cache = False,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

# 5.0 Configure the Tokenizer

In [9]:
# Configure the tokenizer, use left-padding to optimize memory usage during training.
tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          trust_remote_code=True,
                                          padding_side="left",
                                          add_eos_token=True,
                                          add_bos_token=True,
                                          use_fast=False)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# 6.0 Test the Base Model's Response

In [11]:
# Insert prompt
prompt = "You wanted to subtract 46 from a number, but you accidentally subtract 59 and get 43. How much do you get from the correct calculation?"

# Format the prompt
messages = [
    {"role": "system", "content": "You are a helpful math assistant. Provide responses in plain and simple text."},
    {"role": "user", "content": prompt}
]

# Apply chat template
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False, # Keep text as string
    add_generation_prompt=True # Adds additional instructions (if needed)
)

# Tokenize the text
model_inputs = tokenizer([text], return_tensors="pt").to(base_model.device)

# Generate response
generated_ids = base_model.generate(
    **model_inputs,
    max_new_tokens=512
)

# Get the generated tokens
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# Decode the tokens into text
base_response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("BASE MODEL RESPONSE \n============================================== \n", base_response)

BASE MODEL RESPONSE 
 To find out how much you should have gotten if you correctly performed the subtraction, you can use the equation:

\[ \text{Correct Answer} = \text{Number} - 59 - 46 \]

Given that your answer was 43, we can substitute it into the equation:

\[ 43 = \text{Number} - 59 - 46 \]

Solving for the Number:

\[ \text{Number} = 43 + 59 + 46 \]

\[ \text{Number} = 108 \]

So, if you had the correct result of 43, you would need to add 59 and 46 to get back to 108.


# 7.0 Dataset Preprocessing

In [12]:
from functools import partial

def create_prompt_formats(sample):
    # Define the instruction
    instruction = "Below is a question and its corresponding answer about mathematics. "

    # Format the question and answer into the desired prompt
    ques_prompt = f"<QUESTION>: {sample['question']}"
    ans_response = f"<ANSWER>: {sample['answer']}"

    # Combine the instruction, question, and answer into a single formatted text
    formatted_prompt = f"{instruction}\n{ques_prompt}\n{ans_response}"

    # Add the new 'text' key to the sample dictionary
    sample["text"] = formatted_prompt

    return sample

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
# To get the model's maximum token limit
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

# Tokenize each batch
def preprocess_batch(batch, tokenizer, max_length):
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    # Add prompt to each sample
    dataset = dataset.map(create_prompt_formats)#, batched=True)

    # Apply preprocessing to each batch of the dataset & and remove existing fields 'question' and 'answer'
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['question', 'answer'],
    )

    # Filter out rows with input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [13]:
# Set a random seed
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

# Get the max_length
max_length = get_max_length(base_model)
print(max_length)

# Preprocess the train and validation dataset
train_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['train'])
val_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['validation'])

Found max lenth: 32768
32768


Map:   0%|          | 0/160028 [00:00<?, ? examples/s]

Map:   0%|          | 0/160028 [00:00<?, ? examples/s]

Filter:   0%|          | 0/160028 [00:00<?, ? examples/s]

Map:   0%|          | 0/20003 [00:00<?, ? examples/s]

Map:   0%|          | 0/20003 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20003 [00:00<?, ? examples/s]

CPU times: user 6min 45s, sys: 3.75 s, total: 6min 49s
Wall time: 7min 12s


In [13]:
print(train_dataset)

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 160028
})


# 8.0 Fine Tuning

## 8.1 Configure LoRA and Initialize LoRA adapter (LoRA trainable version of the model)

In [14]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Configure the LoRA parameters
config = LoraConfig(
    r=32, # Rank
    lora_alpha=64,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# Enable gradient checkpointing to reduce memory usage during fine-tuning
base_model.gradient_checkpointing_enable()

# Prepare the base model for QLoRA
base_model = prepare_model_for_kbit_training(base_model)

# Get the LoRA trainable version of the model (LoRA adapter)
peft_model = get_peft_model(base_model, config)

# Check the no. of trainable parameters
peft_model.print_trainable_parameters()

trainable params: 2,949,120 || all params: 496,981,888 || trainable%: 0.5934


## 8.2 Define 'TrainingArguments' and Create 'Trainer' Instance

In [15]:
import transformers

# Define the output directory
output_model_name = f'Qwen2.5-Math-Bot-0.5B-{time.strftime("%Y%m%d")}'
output_dir = f'./{output_model_name}'

# Define the training arguments
peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,
    # num_train_epochs=25,
    learning_rate=1e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    # eval_strategy="steps",
    # eval_steps = 100,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="wandb",
    overwrite_output_dir = 'True',
    group_by_length=True,
    # fp16=True,
)

# Disable caching to save memory
peft_model.config.use_cache = False

# Create the 'Trainer' instance
peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

max_steps is given, it will override any value given in num_train_epochs


## 8.3 Start Training

In [16]:
# To save memory
del base_model
del dataset
del train_testvalid
del test_valid
del bnb_config
torch.cuda.empty_cache()

In [17]:
# Start training the model
peft_trainer.train()

# Stop reporting to wandb
wandb.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: Currently logged in as: 0132114 (0132114-uow-malaysia). Use `wandb login --relogin` to force relogin


Step,Training Loss
25,0.690000
50,0.863800
75,0.633800
100,0.712600
125,0.582800
150,0.679900
175,0.553100
200,0.648700
225,0.538200
250,0.652000


train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▃█▁▆▁▇▁▄▃▅▁▄▃▄▂█▁▅▁▄▂▆▂▇▂▅▂▃▂▅▂▅▂▅▂▆▁▆▂▄
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▅█▄▅▃▅▂▄▂▄▂▄▂▄▁▃▃▃▁▄▁▄▁▃▂▃▂▂▂▂▁▃▁▃▁▃▂▃▂▃
total_flos,3211656557608704.0
train/epoch,0.025
train/global_step,1000
train/grad_norm,1.48237
train/learning_rate,0
train/loss,0.6053


In [18]:
# Free memory for merging weights
# del peft_trainer
torch.cuda.empty_cache()

# 9.0 Merge Fine Tuned LoRA Adapter to the Base Model

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Configure Bits and Bytes to load the model in 4-bit (quantized)
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, # Load the weights in 4 bit
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype, # Uses 16-bit floating-point (float16)
        bnb_4bit_use_double_quant=True, # Enable double quantization
    )

base_model_name = 'Qwen/Qwen2.5-0.5B-Instruct'
base_model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [20]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_name, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [21]:
from peft import PeftModel

final_dir = f'/content/{output_model_name}/checkpoint-1000'

# Get the LoRA adapter
ft_model = PeftModel.from_pretrained(base_model, final_dir, torch_dtype=torch.float16, is_trainable=False)

# Merge the LoRA adapter with the base model and save the merged model
lora_merged_model = ft_model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


# 10.0 Test the Fine Tuned Model

In [22]:
# Insert prompt
prompt = "You wanted to subtract 46 from a number, but you accidentally subtract 59 and get 43. How much do you get from the correct calculation?"

# Format the prompt
messages = [
    {"role": "system", "content": "You are a helpful math assistant. Provide responses in plain text. Be as accurate as possible"},
    {"role": "user", "content": prompt}
]

# Apply chat template
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False, # Keep text as string
    add_generation_prompt=True # Adds additional instructions (if needed)
)

# Tokenize the text
model_inputs = tokenizer([text], return_tensors="pt").to(lora_merged_model.device)

# Generate response
generated_ids = lora_merged_model.generate(
    **model_inputs,
    max_new_tokens=512
)

# Get the generated tokens
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# Decode the tokens into text
ft_response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("BASE RESPONSE \n============================================== \n", base_response)
print("\n --------------------------------------------------------------------------------------------------------------------- \n")
print("FINE TUNED RESPONSE \n============================================== \n", ft_response)

BASE RESPONSE 
 To find out how much you should have gotten if you correctly performed the subtraction, you can use the equation:

\[ \text{Correct Answer} = \text{Number} - 59 - 46 \]

Given that your answer was 43, we can substitute it into the equation:

\[ 43 = \text{Number} - 59 - 46 \]

Solving for the Number:

\[ \text{Number} = 43 + 59 + 46 \]

\[ \text{Number} = 108 \]

So, if you had the correct result of 43, you would need to add 59 and 46 to get back to 108.

 --------------------------------------------------------------------------------------------------------------------- 

FINE TUNED RESPONSE 
 To find out how much you would have gotten if you had actually performed the subtraction correctly (subtracting 59 instead of 46), we need to follow these steps:

1. Start with the incorrect result: You got 43 after subtracting 59.
2. To correct this, add 59 to your result: 43 + 59 = 102.

So, you should end up with 102 when performing the correct calculation. The answer is that

# 11.0 Push to HuggingFace

In [23]:
# lora_merged_model.save_pretrained("merged",safe_serialization=True)
# tokenizer.save_pretrained("merged")

# #push merged model to the hub
# lora_merged_model.push_to_hub("Qwen2.5-Math-Bot-0.5B-20241203-1546") # the name of the model you want
# tokenizer.push_to_hub("Qwen2.5-Math-Bot-0.5B-20241203-1546")

In [24]:
from google.colab import runtime
runtime.unassign()